## Loading packages

In [ ]:
import pandas as pd, numpy as np, warnings, matplotlib.pyplot as plt
from pathlib import Path
from typing import Dict, List, Tuple
from itertools import product

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GroupKFold
from sklearn.metrics import (roc_auc_score, average_precision_score, brier_score_loss,
                             roc_curve, precision_recall_curve)
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.inspection import permutation_importance

warnings.filterwarnings("ignore", category=UserWarning)

## Set some inputs

In [ ]:
TRAIN_PATH = Path("dt.csv")  
TEST_PATH  = Path("dt.csv")   
RANDOM_STATE = 42
CALIB_METHOD = "isotonic"  
N_FOLDS_MAX  = 5           
TARGET_SENS  = 0.70
TARGET_SPEC  = 0.70
N_BOOT       = 1000       
np.random.seed(RANDOM_STATE)

WEIGHT_CHOICES = ["WTMEC8YR","WTMEC6YR","WTMEC4YR","WTMEC2YR"]
DESIGN_COLS    = ["SDMVPSU","SDMVSTRA"]
OPTIONAL_DESIGN= ["SDDSRVYR"]
ID_COLS        = ["SEQN","split"]


## Define functions for data processing and model development/validation

In [ ]:

def ensure_outcome(df: pd.DataFrame) -> pd.DataFrame:
    if "dep01" in df.columns:
        out = df.copy()
    elif "dep" in df.columns:
        y = df["dep"].astype(str).str.lower().map({"yes":1,"no":0})
        out = df.copy(); out["dep01"] = y
    else:
        raise ValueError("Outcome not found: need 'dep01' or 'dep'.")
    out = out[~out["dep01"].isna()].copy()
    out["dep01"] = out["dep01"].astype(int)
    return out

def minimal_clean(df: pd.DataFrame) -> pd.DataFrame:
    df = ensure_outcome(df)
    for col in DESIGN_COLS:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")
    return df.dropna(subset=DESIGN_COLS)

def infer_n_cycles(df: pd.DataFrame) -> int:
    return int(pd.Series(df["SDDSRVYR"]).nunique()) if "SDDSRVYR" in df.columns else 1

def get_mec_weights(df: pd.DataFrame) -> np.ndarray:
    for c in ["WTMEC8YR","WTMEC6YR","WTMEC4YR"]:
        if c in df.columns:
            w = df[c].to_numpy(dtype=float)
            return np.where(np.isfinite(w), w, 0.0)
    if "WTMEC2YR" in df.columns:
        ncyc = max(1, infer_n_cycles(df))
        w = (df["WTMEC2YR"] / float(ncyc)).to_numpy(dtype=float)
        return np.where(np.isfinite(w), w, 0.0)
    raise ValueError("No MEC weight found (WTMEC{2,4,6,8}YR).")

def build_groups(df: pd.DataFrame) -> np.ndarray:
    if "SDDSRVYR" in df.columns:
        grp = (df["SDDSRVYR"].astype(str) + "_" +
               df["SDMVSTRA"].astype(str) + "_" +
               df["SDMVPSU"].astype(str)).to_numpy()
    else:
        grp = (df["SDMVSTRA"].astype(str) + "_" +
               df["SDMVPSU"].astype(str)).to_numpy()
    return grp


import re
EXACT_DROP_RAW = {"SDMVPSU","SDMVSTRA","SDDSRVYR","SEQN","SPLIT","STRAT","PSU"}
STARTS_WITH = ("WT", "WTS", "WTSA", "SDMV", "SDDS")     
CONTAINS    = ("STRAT", "PSU")                          

def _norm(name: str) -> str:
    return str(name).strip().upper()

EXACT_DROP = {_norm(x) for x in EXACT_DROP_RAW}

def _is_excluded(col: str) -> bool:
    c = col.upper()
    if c in EXACT_DROP: 
        return True
    if c.startswith(STARTS_WITH): 
        return True
    return any(token in c for token in CONTAINS)

def infer_feature_cols(df: pd.DataFrame) -> tuple[list[str], list[str]]:
    cols = [c for c in df.columns if c != "dep01" and not _is_excluded(c)]
    num_cols = [c for c in cols if pd.api.types.is_numeric_dtype(df[c])]
    cat_cols = [c for c in cols if c not in num_cols]
    return num_cols, cat_cols


from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd

def _is_binary_series(s: pd.Series) -> bool:
    u = pd.unique(s[~pd.isna(s)])
    return len(u) <= 2 and set(pd.Series(u).dropna().astype(float)).issubset({0.0, 1.0})

def make_preprocessor(num_cols: list[str], cat_cols: list[str], df_fit: pd.DataFrame) -> ColumnTransformer:
    bin_cols = [c for c in num_cols if _is_binary_series(df_fit[c])]
    num_nonbin = [c for c in num_cols if c not in bin_cols]

    num_pipe = Pipeline([
        ("imp", SimpleImputer(strategy="median")),
        ("sc",  StandardScaler())
    ])
    bin_pipe = Pipeline([
        ("imp", SimpleImputer(strategy="most_frequent"))  
    ])
    cat_pipe = Pipeline([
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("oh",  OneHotEncoder(handle_unknown="ignore", sparse=False))
    ])

    return ColumnTransformer([
        ("num", num_pipe, num_nonbin),
        ("bin", bin_pipe, bin_cols),
        ("cat", cat_pipe, cat_cols)
    ], remainder="drop", sparse_threshold=0.0)


def feature_names_out(prep: ColumnTransformer) -> List[str]:
    try:
        return list(prep.get_feature_names_out())
    except Exception:
        num_names = list(prep.transformers_[0][2])
        oh = prep.transformers_[1][1]["oh"]
        base = list(prep.transformers_[1][2])
        cat_names = []
        if hasattr(oh, "categories_"):
            for col, cats in zip(base, oh.categories_):
                for c in cats:
                    cat_names.append(f"{col}_{c}")
        else:
            cat_names = base
        return num_names + cat_names



def model_factories():
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.naive_bayes import GaussianNB

    grids = {}

    lr = LogisticRegression(penalty="elasticnet", solver="saga",
                            max_iter=10000, random_state=RANDOM_STATE)
    grids["LR"] = (lr, {
        "classifier__C": [0.1, 1.0],
        "classifier__l1_ratio": [0.25, 0.5, 0.75],
    })


    rf = RandomForestClassifier(n_estimators=100, 
                                n_jobs=-1, random_state=RANDOM_STATE)
    grids["RF"] = (rf, {
        "classifier__n_estimators": [100, 
                                     200, 500, 
                                     800, 1000
                                    ],
        "classifier__min_samples_leaf": [5, 10],
        "classifier__max_depth": [5 ,10]
    })

    nb = GaussianNB()
    grids["NB"] = (nb, {})  
    
    dt = DecisionTreeClassifier(random_state=42)
    grids["DT"] = (dt, {
        "classifier__criterion": ["gini", "entropy"],
        "classifier__max_depth": [None, 5, 8, 12],
        "classifier__min_samples_split": [2, 5, 10],
        "classifier__min_samples_leaf": [1, 5, 10],
        "classifier__ccp_alpha": [0.0, 0.0005, 0.001]
                                  })
    return grids


def metrics_weighted(y, p, w):
    auc = roc_auc_score(y, p, sample_weight=w)           
    ap  = average_precision_score(y, p, sample_weight=w) 
    b   = brier_score_loss(y, p, sample_weight=w)        
    return auc, ap, b

def confusion_at(y, p, w, thr):
    yhat = (p >= thr).astype(int)
    pos = (y == 1); neg = ~pos
    tp = float(np.sum(w[(yhat==1) & pos])); fn = float(np.sum(w[(yhat==0) & pos]))
    tn = float(np.sum(w[(yhat==0) & neg])); fp = float(np.sum(w[(yhat==1) & neg]))
    sens = tp/(tp+fn) if tp+fn>0 else np.nan
    spec = tn/(tn+fp) if tn+fp>0 else np.nan
    ppv  = tp/(tp+fp) if tp+fp>0 else np.nan
    npv  = tn/(tn+fn) if tn+fn>0 else np.nan
    prev = float(np.sum(w[pos]))/float(np.sum(w)) if np.sum(w)>0 else np.nan
    return {"threshold":float(thr),"sens":sens,"spec":spec,"ppv":ppv,"npv":npv,"prevalence":prev}

def pick_threshold_constraints(y, p, w, sens_min=0.70, spec_min=0.70):
    fpr, tpr, thr = roc_curve(y, p, sample_weight=w)
    spec = 1.0 - fpr
    feasible = (tpr >= sens_min) & (spec >= spec_min)
    if np.any(feasible):
        idx = np.argmax(tpr * feasible)     
        return confusion_at(y, p, w, thr[idx])
    mm = np.minimum(tpr, spec)
    idx = int(np.argmax(mm))
    return confusion_at(y, p, w, thr[idx])

def fit_with_weights(pipe, X, y, w):
    try:
        pipe.fit(X, y, **{"classifier__sample_weight": w})
    except TypeError:
        pipe.fit(X, y)

def group_cv_weighted_auc(pipe, params, X, y, w, groups, gkf) -> float:
    aucs = []
    for tr, va in gkf.split(X, y, groups=groups):
        Xtr, Xva = X.iloc[tr], X.iloc[va]
        ytr, yva = y[tr], y[va]
        wtr, wva = w[tr], w[va]
        pipe.set_params(**params)
        fit_with_weights(pipe, Xtr, ytr, wtr)
        p = pipe.predict_proba(Xva)[:,1]
        aucs.append(roc_auc_score(yva, p, sample_weight=wva))
    return float(np.mean(aucs))

def oof_predict(pipe, params, X, y, w, groups, gkf):
    p = np.zeros_like(y, dtype=float); order = np.zeros_like(y, dtype=int)
    fold_id = 0
    for tr, va in gkf.split(X, y, groups=groups):
        Xtr, Xva = X.iloc[tr], X.iloc[va]
        ytr, yva = y[tr], y[va]
        wtr = w[tr]
        pipe.set_params(**params)
        fit_with_weights(pipe, Xtr, ytr, wtr)
        p[va] = pipe.predict_proba(Xva)[:,1]
        order[va] = fold_id
        fold_id += 1
    return p, order



## Load data

In [ ]:


def load_dataset(path: Path) -> pd.DataFrame:
    return minimal_clean(pd.read_csv(path))

df_tr = load_dataset(TRAIN_PATH)
df_te = load_dataset(TEST_PATH)
if df_te.empty:
    raise RuntimeError("Test set has 0 rows after filtering/split.")

y_tr = df_tr["dep01"].to_numpy(int)
y_te = df_te["dep01"].to_numpy(int)

w_tr = get_mec_weights(df_tr)
w_te = get_mec_weights(df_te)

groups_tr = build_groups(df_tr)
psu_te    = df_te["SDMVPSU"].astype(str).to_numpy()  
strat_te  = df_te["SDMVSTRA"].astype(str).to_numpy()

num_cols, cat_cols = infer_feature_cols(df_tr)
preproc = make_preprocessor(num_cols, cat_cols, df_tr)

X_tr = df_tr[num_cols + cat_cols]
X_te = df_te[num_cols + cat_cols]


## Tune and evaluate models

In [ ]:

factories = model_factories()
n_groups = len(np.unique(groups_tr))
if n_groups < 2:
    raise ValueError(f"Grouped CV needs >=2 groups; got {n_groups}.")
n_folds = max(2, min(N_FOLDS_MAX, n_groups))
gkf = GroupKFold(n_splits=n_folds)

cv_summary = []
oof_store  = {}  

best_name, best_auc, best_params, best_pipe = None, -np.inf, None, None

for name, (est, grid) in factories.items():
    pipe = Pipeline([("prep", preproc), ("classifier", est)])
    if not grid:
        auc_cv, params = group_cv_weighted_auc(pipe, {}, X_tr, y_tr, w_tr, groups_tr, gkf), {}
    else:
        scores = []
        keys, vals = zip(*grid.items())
        for combo in product(*vals):
            params = dict(zip(keys, combo))
            aucs = group_cv_weighted_auc(pipe, params, X_tr, y_tr, w_tr, groups_tr, gkf)
            scores.append((aucs, params))
        auc_cv, params = max(scores, key=lambda t: t[0])
    p_oof, fold_id = oof_predict(pipe, params, X_tr, y_tr, w_tr, groups_tr, gkf)
    oof_store[name] = p_oof

    auc, ap, brier = metrics_weighted(y_tr, p_oof, w_tr)
    thr_rule = pick_threshold_constraints(y_tr, p_oof, w_tr, TARGET_SENS, TARGET_SPEC)
    row = {"model":name, "cv_auc":auc_cv, "oof_auc":auc, "oof_prauc":ap,
           "oof_brier":brier, "oof_thr":thr_rule["threshold"],
           "oof_sens":thr_rule["sens"], "oof_spec":thr_rule["spec"],
           "oof_ppv":thr_rule["ppv"], "oof_npv":thr_rule["npv"]}
    cv_summary.append(row)
    print(f"[CV] {name}: mean AUC={auc_cv:.3f} | OOF AUC={auc:.3f} | OOF PR-AUC={ap:.3f} | Brier={brier:.3f}")

    if auc_cv > best_auc:
        best_name, best_auc, best_params = name, auc_cv, params
        best_pipe = Pipeline([("prep", preproc), ("classifier", est)])
        if best_params: best_pipe.set_params(**best_params)

cv_df = pd.DataFrame(cv_summary).sort_values("oof_auc", ascending=False)
print("\n=== TRAIN OOF summary (all tuned models) ===")
print(cv_df.round(3))


## PSU bootstrap for train and test

In [ ]:
import numpy as np, pandas as pd
from collections import Counter
from scipy.special import expit, logit
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss

EPS = 1e-6  

def _clip01(x, eps=EPS):
    x = np.asarray(x, float)
    return np.clip(x, eps, 1.0 - eps)

def ci_percentile(x, lo=2.5, hi=97.5):
    x = np.asarray(x, float)
    return (float(np.nanpercentile(x, lo)), float(np.nanpercentile(x, hi)))

def ci_logit_percentile(x, lo=2.5, hi=97.5):
    z = logit(_clip01(x))              
    lz, hz = np.nanpercentile(z, [lo, hi])
    return (float(expit(lz)), float(expit(hz)))  

def metrics_weighted(y, p, w):
    return (
        roc_auc_score(y, p, sample_weight=w),
        average_precision_score(y, p, sample_weight=w),
        brier_score_loss(y, p, sample_weight=w),
    )

def confusion_at(y, p, w, thr):
    y = np.asarray(y, int); p = np.asarray(p, float); w = np.asarray(w, float)
    yhat = (p >= float(thr)).astype(int)
    pos = (y == 1); neg = ~pos
    tp = float(np.sum(w[(yhat==1) & pos])); fn = float(np.sum(w[(yhat==0) & pos]))
    tn = float(np.sum(w[(yhat==0) & neg])); fp = float(np.sum(w[(yhat==1) & neg]))
    sens = tp/(tp+fn) if (tp+fn)>0 else np.nan
    spec = tn/(tn+fp) if (tn+fp)>0 else np.nan
    ppv  = tp/(tp+fp) if (tp+fp)>0 else np.nan
    npv  = tn/(tn+fn) if (tn+fn)>0 else np.nan
    return {"sens": sens, "spec": spec, "ppv": ppv, "npv": npv}



def psu_multiplicity_unstratified(psu_ids, B=1000, seed=42):
    rng = np.random.RandomState(seed)
    u = np.unique(psu_ids)
    for _ in range(B):
        draw = rng.choice(u, size=len(u), replace=True)
        cnt = Counter(draw)
        yield np.vectorize(cnt.get)(psu_ids, 0).astype(float)

def psu_multiplicity_within_strata(psu_ids, strata_ids, B=1000, seed=42):
    rng = np.random.RandomState(seed)
    psu_ids = np.asarray(psu_ids)
    strata_ids = np.asarray(strata_ids)
    u_strata = np.unique(strata_ids)
    n = len(psu_ids)
    for _ in range(B):
        mult = np.zeros(n, float)
        for s in u_strata:
            m = (strata_ids == s)
            psu_s = np.unique(psu_ids[m])
            draw = rng.choice(psu_s, size=len(psu_s), replace=True)
            cnt = Counter(draw)
            mult[m] = np.vectorize(cnt.get)(psu_ids[m], 0).astype(float)
        yield mult

def choose_psu_generator(psu_ids, strata_ids=None, B=1000, seed=42):
    if strata_ids is None:
        print("No strata provided: using unstratified PSU bootstrap.")
        return psu_multiplicity_unstratified(psu_ids, B=B, seed=seed)
    npsu = pd.Series(psu_ids).groupby(pd.Series(strata_ids)).nunique()
    if (npsu >= 2).all():
        return psu_multiplicity_within_strata(psu_ids, strata_ids, B=B, seed=seed)
    else:
        print("WARNING: some strata have only 1 PSU → using UNSTRATIFIED PSU bootstrap for CIs.")
        return psu_multiplicity_unstratified(psu_ids, B=B, seed=seed)


def bootstrap_metrics_with_counts(
    y, p, w, thr, psu_ids, strata_ids=None, B=1000, seed=42,
    ci_method="logit"  
):
    y = np.asarray(y, int); p = np.asarray(p, float); w = np.asarray(w, float)
    gen = choose_psu_generator(psu_ids, strata_ids, B=B, seed=seed)

    aucs, aps, brs, sens, specs, ppvs, npvs = ([] for _ in range(7))
    for mult in gen:
        wb = w * mult
        if not np.isfinite(wb).any() or wb.sum() == 0:
            continue
        if (np.sum(wb[y==1]) == 0) or (np.sum(wb[y==0]) == 0):
            continue
        a  = roc_auc_score(y, p, sample_weight=wb)
        ap = average_precision_score(y, p, sample_weight=wb)
        b  = brier_score_loss(y, p, sample_weight=wb)
        cm = confusion_at(y, p, wb, thr)
        aucs.append(a); aps.append(ap); brs.append(b)
        sens.append(cm["sens"]); specs.append(cm["spec"])
        ppvs.append(cm["ppv"]);  npvs.append(cm["npv"])

    if ci_method == "logit":
        ci_bounded = ci_logit_percentile
    elif ci_method == "percentile":
        ci_bounded = ci_percentile
    else:
        raise ValueError("ci_method must be 'logit' or 'percentile'.")

    return {
        "auc_ci"  : ci_bounded(aucs),
        "prauc_ci": ci_bounded(aps),
        "brier_ci": ci_bounded(brs),
        "sens_ci" : ci_bounded(sens),
        "spec_ci" : ci_bounded(specs),
        "ppv_ci"  : ci_bounded(ppvs),
        "npv_ci"  : ci_bounded(npvs),
        "n_reps"  : len(aucs)
    }


N_BOOT = 1000
RANDOM_STATE = 42

psu_tr  = df_tr["SDMVPSU"].astype(str).to_numpy()
stra_tr = df_tr["SDMVSTRA"].astype(str).to_numpy() if "SDMVSTRA" in df_tr.columns else None

train_ci_rows = []
for name, p_oof in oof_store.items():
    thr_m = float(cv_df.loc[cv_df["model"]==name, "oof_thr"].iloc[0])
    auc, ap, brier = metrics_weighted(y_tr, p_oof, w_tr)
    cm = confusion_at(y_tr, p_oof, w_tr, thr_m)
    ci = bootstrap_metrics_with_counts(
        y_tr, p_oof, w_tr, thr_m, psu_tr, strata_ids=stra_tr,
        B=N_BOOT, seed=RANDOM_STATE, ci_method="logit"
    )
    train_ci_rows.append({
        "model": name,
        "oof_auc": auc,     "oof_auc_lo": ci["auc_ci"][0],   "oof_auc_hi": ci["auc_ci"][1],
        "oof_prauc": ap,    "oof_prauc_lo": ci["prauc_ci"][0], "oof_prauc_hi": ci["prauc_ci"][1],
        "oof_brier": brier, "oof_brier_lo": ci["brier_ci"][0], "oof_brier_hi": ci["brier_ci"][1],
        "oof_thr": thr_m,
        "oof_sens": cm["sens"], "oof_sens_lo": ci["sens_ci"][0], "oof_sens_hi": ci["sens_ci"][1],
        "oof_spec": cm["spec"], "oof_spec_lo": ci["spec_ci"][0], "oof_spec_hi": ci["spec_ci"][1],
        "oof_ppv": cm["ppv"],   "oof_ppv_lo": ci["ppv_ci"][0],   "oof_ppv_hi": ci["ppv_ci"][1],
        "oof_npv": cm["npv"],   "oof_npv_lo": ci["npv_ci"][0],   "oof_npv_hi": ci["npv_ci"][1],
        "oof_boot_reps": ci["n_reps"]
    })

train_oof_ci_df = pd.DataFrame(train_ci_rows).sort_values("oof_auc", ascending=False)
print("\n=== TRAIN (OOF) PSU-bootstrap 95% CIs — all tuned models (logit-percentile) ===")
print(train_oof_ci_df.round(3)[[
    "model",
    "oof_auc","oof_auc_lo","oof_auc_hi",
    "oof_prauc","oof_prauc_lo","oof_prauc_hi",
    "oof_brier","oof_brier_lo","oof_brier_hi",
    "oof_sens","oof_sens_lo","oof_sens_hi",
    "oof_spec","oof_spec_lo","oof_spec_hi",
    "oof_ppv","oof_ppv_lo","oof_ppv_hi",
    "oof_npv","oof_npv_lo","oof_npv_hi",
    "oof_boot_reps"
]])


train_oof_ci_df.to_csv("train_oof_psu_bootstrap_cis.csv", index=False)
print("\nSaved: train_oof_psu_bootstrap_cis.csv")

## Test the best model on the testing set

In [ ]:

def fit_with_weights_full(pipe, X, y, w):
    try:
        pipe.fit(X, y, **{"classifier__sample_weight": w})
    except TypeError:
        pipe.fit(X, y)

fit_with_weights_full(best_pipe, X_tr, y_tr, w_tr)
p_te = best_pipe.predict_proba(X_te)[:,1]

thr_train_best = float(cv_df.loc[cv_df["model"]==best_name, "oof_thr"].iloc[0])

auc_te, ap_te, brier_te = metrics_weighted(y_te, p_te, w_te)
thr_rule_te = confusion_at(y_te, p_te, w_te, thr_train_best)

print(f"\n=== TEST (best model: {best_name}) ===")
print(f"ROC-AUC : {auc_te:.3f}")
print(f"PR-AUC  : {ap_te:.3f}")
print(f"Brier   : {brier_te:.3f}")
print(f"Thr({thr_train_best:.4f}) Sens/Spec = {thr_rule_te['sens']:.3f}/{thr_rule_te['spec']:.3f} | "
      f"PPV/NPV = {thr_rule_te['ppv']:.3f}/{thr_rule_te['npv']:.3f}")

## Recalibration

In [ ]:

try:
    cal = CalibratedClassifierCV(best_pipe, method=CALIB_METHOD, cv=min(5, n_groups))
    try:
        cal.fit(X_tr, y_tr, **{"classifier__sample_weight": w_tr})
    except TypeError:
        cal.fit(X_tr, y_tr)
    p_te_cal = cal.predict_proba(X_te)[:,1]
    auc_cal, ap_cal, brier_cal = metrics_weighted(y_te, p_te_cal, w_te)
    thr_rule_te_cal = confusion_at(y_te, p_te_cal, w_te, thr_train_best)  
    print(f"\n=== TEST after {CALIB_METHOD} calibration ===")
    print(f"ROC-AUC : {auc_cal:.3f}")
    print(f"PR-AUC  : {ap_cal:.3f}")
    print(f"Brier   : {brier_cal:.3f}")
    print(f"Thr({thr_train_best:.4f}) Sens/Spec = {thr_rule_te_cal['sens']:.3f}/{thr_rule_te_cal['spec']:.3f} | "
          f"PPV/NPV = {thr_rule_te_cal['ppv']:.3f}/{thr_rule_te_cal['npv']:.3f}")
except Exception as e:
    print("Calibration skipped:", e)
    p_te_cal = None

## PLOT

In [ ]:

def plot_train_oof_curves(oof_store, y, w, title_prefix="TRAIN"):
    plt.figure(figsize=(7,6))
    for name, p in oof_store.items():
        fpr, tpr, _ = roc_curve(y, p, sample_weight=w)
        auc = roc_auc_score(y, p, sample_weight=w)
        plt.plot(fpr, tpr, label=f"{name} (AUC={auc:.3f})")
    plt.plot([0,1],[0,1],'--',alpha=0.5)
    plt.xlabel("1 - Specificity"); plt.ylabel("Sensitivity")
    plt.title(f"{title_prefix} ROC")
    plt.legend(); plt.tight_layout(); plt.savefig("train_oof_roc.png", dpi=150)

    prev = float(np.sum(w[(y==1)]) / np.sum(w)) if np.sum(w)>0 else 0.0
    plt.figure(figsize=(7,6))
    for name, p in oof_store.items():
        prec, rec, _ = precision_recall_curve(y, p, sample_weight=w)
        ap = average_precision_score(y, p, sample_weight=w)
        plt.plot(rec, prec, label=f"{name} (PRAUC={ap:.3f})")
    plt.axhline(prev, linestyle="--", alpha=0.7, label=f"Baseline (prev={prev:.3f})")
    plt.xlabel("Recall"); plt.ylabel("Precision")
    plt.title(f"{title_prefix} PR")
    plt.legend(); plt.tight_layout(); plt.savefig("train_oof_pr.png", dpi=150)


def plot_test_best_curves(y, p, w, p_cal=None, title_prefix="TEST"):
    plt.figure(figsize=(7,6))
    fpr, tpr, _ = roc_curve(y, p, sample_weight=w)
    plt.plot(fpr, tpr, label=f"Best (AUC={roc_auc_score(y,p,sample_weight=w):.3f})")
    if p_cal is not None:
        fpr2, tpr2, _ = roc_curve(y, p_cal, sample_weight=w)
        plt.plot(fpr2, tpr2, label=f"Best + calib (AUC={roc_auc_score(y,p_cal,sample_weight=w):.3f})")
    plt.plot([0,1],[0,1],'--',alpha=0.5)
    plt.xlabel("1 - Specificity"); plt.ylabel("Sensitivity")
    plt.title(f"{title_prefix} ROC"); plt.legend(); plt.tight_layout(); plt.savefig("test_best_roc.png", dpi=150)

    prev = float(np.sum(w[(y==1)]) / np.sum(w)) if np.sum(w)>0 else 0.0
    plt.figure(figsize=(7,6))
    pr, rc, _ = precision_recall_curve(y, p, sample_weight=w)
    plt.plot(rc, pr, label=f"Best (AP={average_precision_score(y,p,sample_weight=w):.3f})")
    if p_cal is not None:
        pr2, rc2, _ = precision_recall_curve(y, p_cal, sample_weight=w)
        plt.plot(rc2, pr2, label=f"Best + calib (PRAUC={average_precision_score(y,p_cal,sample_weight=w):.3f})")
    plt.axhline(prev, linestyle="--", alpha=0.7, label=f"Baseline (prev={prev:.3f})")
    plt.xlabel("Recall"); plt.ylabel("Precision")
    plt.title(f"{title_prefix} PR"); plt.legend(); plt.tight_layout(); plt.savefig("test_best_pr.png", dpi=150)

plot_train_oof_curves(oof_store, y_tr, w_tr)
plot_test_best_curves(y_te, p_te, w_te, p_te_cal)


In [ ]:

import numpy as np, matplotlib.pyplot as plt
from typing import Tuple
from sklearn.metrics import roc_auc_score, brier_score_loss, average_precision_score

def _weighted_quantile(x: np.ndarray, w: np.ndarray, q: np.ndarray) -> np.ndarray:
    order = np.argsort(x)
    x_sorted, w_sorted = x[order], w[order]
    cdf = np.cumsum(w_sorted) / np.sum(w_sorted)
    return np.interp(q, cdf, x_sorted)

def weighted_calibration_curve(
    y: np.ndarray,
    p: np.ndarray,
    w: np.ndarray,
    n_bins: int = 10,
    strategy: str = "quantile",
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    y = np.asarray(y).astype(int)
    p = np.asarray(p).astype(float)
    w = np.asarray(w).astype(float)
    mask = np.isfinite(p) & np.isfinite(w)
    y, p, w = y[mask], p[mask], np.clip(w[mask], 0.0, np.inf)


    if strategy == "quantile":
        qs = np.linspace(0.0, 1.0, n_bins + 1)
        edges = _weighted_quantile(p, w, qs)
        edges[0], edges[-1] = 0.0, 1.0
    else:
        edges = np.linspace(0.0, 1.0, n_bins + 1)

    edges = np.unique(edges)
    if len(edges) - 1 < 2:
        edges = np.linspace(0.0, 1.0, min(n_bins, 10) + 1)

    bin_idx = np.digitize(p, edges[1:-1], right=False)
    pred_means, true_means, bin_w = [], [], []
    for b in range(len(edges) - 1):
        m = bin_idx == b
        wb = w[m]
        if wb.sum() <= 0:
            continue
        pb = p[m]
        yb = y[m]
        pred_means.append(np.average(pb, weights=wb))
        true_means.append(np.average(yb, weights=wb))
        bin_w.append(wb.sum())

    pred_means = np.asarray(pred_means)
    true_means = np.asarray(true_means)
    bin_w = np.asarray(bin_w)
    return pred_means, true_means, edges, bin_w

def expected_calibration_error(
    y: np.ndarray, p: np.ndarray, w: np.ndarray, n_bins: int = 10
) -> float:
    pred_b, true_b, _, bin_w = weighted_calibration_curve(y, p, w, n_bins=n_bins, strategy="quantile")
    if bin_w.sum() == 0 or len(bin_w) == 0:
        return np.nan
    return float(np.sum((bin_w / bin_w.sum()) * np.abs(true_b - pred_b)))

def _metrics(y, p, w):
    return (roc_auc_score(y, p, sample_weight=w),
            average_precision_score(y, p, sample_weight=w),
            brier_score_loss(y, p, sample_weight=w),
            expected_calibration_error(y, p, w, n_bins=10))

def plot_calibration_test(y, p, w, p_cal=None, model_name="Best", fname_prefix="test_calibration"):
    auc0, ap0, brier0, ece0 = _metrics(y, p, w)
    pred0, true0, edges0, bw0 = weighted_calibration_curve(y, p, w, n_bins=10, strategy="quantile")

    if p_cal is not None:
        auc1, ap1, brier1, ece1 = _metrics(y, p_cal, w)
        pred1, true1, edges1, bw1 = weighted_calibration_curve(y, p_cal, w, n_bins=10, strategy="quantile")
    else:
        auc1 = ap1 = brier1 = ece1 = None
        pred1 = true1 = None

    plt.figure(figsize=(7,6))
    plt.plot([0,1], [0,1], "--", lw=1, label="Perfectly calibrated")
    plt.plot(pred0, true0, "o-", label=f"{model_name} (AUC={auc0:.3f}, ECE={ece0:.3f}, Brier={brier0:.3f})")
    if pred1 is not None:
        plt.plot(pred1, true1, "o-", label=f"{model_name} + calib (AUC={auc1:.3f}, ECE={ece1:.3f}, Brier={brier1:.3f})")
    plt.xlabel("Mean predicted probability")
    plt.ylabel("Observed positive fraction")
    plt.title("Calibration — TEST")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{fname_prefix}_reliability.png", dpi=150)


    bins = np.linspace(0, 1, 21)
    plt.figure(figsize=(7,3.5))
    plt.hist(p, bins=bins, weights=w, alpha=0.6, label="pre-cal", edgecolor="none")
    if p_cal is not None:
        plt.hist(p_cal, bins=bins, weights=w, alpha=0.6, label="post-cal", edgecolor="none")
    plt.xlim(0,1)
    plt.xlabel("Predicted probability")
    plt.ylabel("Weighted count")
    plt.title("Predicted probability distribution — TEST")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{fname_prefix}_hist.png", dpi=150)

    print("\n=== TEST calibration metrics ===")
    print(f"Pre-cal  : AUC={auc0:.3f} | PR-AUC={ap0:.3f} | Brier={brier0:.3f} | ECE={ece0:.3f}")
    if p_cal is not None:
        print(f"Post-cal : AUC={auc1:.3f} | PR-AUC={ap1:.3f} | Brier={brier1:.3f} | ECE={ece1:.3f}")
    print(f"Saved: {fname_prefix}_reliability.png, {fname_prefix}_hist.png")

plot_calibration_test(y_te, p_te, w_te, p_cal=(p_te_cal if 'p_te_cal' in globals() else None),
                      model_name=best_name, fname_prefix="test_calibration")


## Feature importance ranking

In [ ]:
import numpy as np
import pandas as pd

def model_native_importance(fitted_pipe, prep):
    names = list(prep.get_feature_names_out())  
    clf = fitted_pipe.named_steps["classifier"]

    if hasattr(clf, "feature_importances_"):
        imp = np.asarray(clf.feature_importances_, dtype=float)
        if imp.shape[0] != len(names):
            raise ValueError(f"Length mismatch: {imp.shape[0]} importances vs {len(names)} features.")
        return pd.DataFrame({"feature": names, "importance": imp, "source": "model"})

    if hasattr(clf, "coef_"):
        coef = np.asarray(clf.coef_, dtype=float)  
        if coef.ndim == 2 and coef.shape[0] > 1:
            coef_use = np.mean(np.abs(coef), axis=0)
        else:
            coef_use = np.abs(coef).ravel()
        if coef_use.shape[0] != len(names):
            raise ValueError(f"Length mismatch: {coef_use.shape[0]} coefs vs {len(names)} features.")
        return pd.DataFrame({"feature": names, "importance": coef_use, "source": "|coef|"})

    return pd.DataFrame(columns=["feature","importance","source"])

from sklearn.metrics import roc_auc_score
from sklearn.inspection import permutation_importance

def permutation_importance_weighted(fitted_pipe, X, y, w, n_repeats=10, random_state=42):
    base = roc_auc_score(y, fitted_pipe.predict_proba(X)[:, 1], sample_weight=w)
    rng = np.random.RandomState(random_state)
    rows = []
    for col in X.columns:
        scores = []
        for _ in range(n_repeats):
            Xp = X.copy()
            Xp[col] = Xp[col].sample(frac=1.0, random_state=rng).to_numpy()
            p = fitted_pipe.predict_proba(Xp)[:, 1]
            s = roc_auc_score(y, p, sample_weight=w)
            scores.append(base - s)
        rows.append((col, float(np.mean(scores))))
    return pd.DataFrame(rows, columns=["feature","importance"]).sort_values("importance", ascending=False)


def get_transformed_feature_names(prep):
    return list(prep.get_feature_names_out())

def build_transformed_to_raw_map(prep, cat_cols):
    tnames = get_transformed_feature_names(prep)
    mapping = {}

    for t in tnames:
        if t.startswith("num__"):
            mapping[t] = t.split("__", 1)[1]
        elif t.startswith("bin__"):
            mapping[t] = t.split("__", 1)[1]
        elif t.startswith("cat__"):
            rest = t.split("__", 1)[1]
            base = None
            for col in cat_cols:
                prefix = f"{col}_"
                if rest.startswith(prefix):
                    base = col
                    break
            mapping[t] = base if base is not None else rest.split("_", 1)[0]
        else:
            mapping[t] = t
    return mapping

prep = best_pipe.named_steps["prep"]          
trans2raw = build_transformed_to_raw_map(prep, cat_cols)


native_imp = model_native_importance(best_pipe, prep)  
if not native_imp.empty:
    native_imp = native_imp.rename(columns={"feature":"transformed"})
    native_imp["raw_feature"] = native_imp["transformed"].map(trans2raw)
    native_imp["source"] = native_imp.get("source", "model")
    native_imp = native_imp[["raw_feature","importance","source"]]


perm_imp = permutation_importance_weighted(best_pipe, X_te, y_te, w_te, n_repeats=10)  
perm_imp = perm_imp.rename(columns={"feature":"raw_feature"})
perm_imp["source"] = "perm_auc"


feat_combined = pd.concat([native_imp, perm_imp], ignore_index=True)


feat_rank_raw = (feat_combined.groupby("raw_feature", as_index=False)["importance"]
                 .mean()
                 .sort_values("importance", ascending=False))

print("\n=== Unified feature ranking (raw columns; no duplicates) — top 20 ===")
print(feat_rank_raw.head(20))
feat_rank_raw.to_csv("feature_importance_raw.csv", index=False)


def plot_feature_importance(feat_df, top=20, filename="feature_importance_top20_raw.png",
                            title="Feature importance"):
    topk = (feat_df.sort_values("importance", ascending=False)
                    .head(top)
                    .iloc[::-1])  
    plt.figure(figsize=(8, max(6, int(0.35*len(topk)))))
    plt.barh(topk["raw_feature"], topk["importance"])
    plt.xlabel("Importance")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(filename, dpi=150)

plot_feature_importance(feat_rank_raw, top=20)



In [ ]:
psu_te  = df_te["SDMVPSU"].astype(str).to_numpy()
stra_te = df_te["SDMVSTRA"].astype(str).to_numpy() if "SDMVSTRA" in df_te.columns else None


if 'p_te' not in globals():
    p_te = best_pipe.predict_proba(X_te)[:, 1]

if 'thr_train_best' not in globals():
    thr_train_best = float(cv_df.loc[cv_df["model"] == best_name, "oof_thr"].iloc[0])


ci_test = bootstrap_metrics_with_counts(
    y_te, p_te, w_te, thr_train_best, psu_te, strata_ids=stra_te,
    B=N_BOOT, seed=RANDOM_STATE, ci_method="logit"
)
print("\n=== TEST (best) PSU-bootstrap 95% CIs — uncalibrated (logit-percentile) ===")
for k in ["auc_ci","prauc_ci","brier_ci","sens_ci","spec_ci","ppv_ci","npv_ci"]:
    lo, hi = ci_test[k]
    print(f"{k}: [{lo:.3f}, {hi:.3f}]  (reps used: {ci_test['n_reps']})")

if 'p_te_cal' in globals() and p_te_cal is not None:
    ci_test_cal = bootstrap_metrics_with_counts(
        y_te, p_te_cal, w_te, thr_train_best, psu_te, strata_ids=stra_te,
        B=N_BOOT, seed=RANDOM_STATE, ci_method="logit"
    )
    print("\n=== TEST (best) PSU-bootstrap 95% CIs — calibrated (logit-percentile) ===")
    for k in ["auc_ci","prauc_ci","brier_ci","sens_ci","spec_ci","ppv_ci","npv_ci"]:
        lo, hi = ci_test_cal[k]
        print(f"{k}: [{lo:.3f}, {hi:.3f}]  (reps used: {ci_test_cal['n_reps']})")

In [ ]:
cv_df.to_csv("train_oof_model_summary.csv", index=False)
feat_rank_raw.head(50).to_csv("feature_importance_top50.csv", index=False)
print("\nSaved: train_oof_model_summary.csv, feature_importance_top50.csv, "
      "train_oof_roc.png, train_oof_pr.png, test_best_roc.png, test_best_pr.png")


## Save model to compare

In [ ]:
import numpy as np
import pandas as pd


df_te = df_te.reset_index(drop=True)


p_out = p_te_cal if 'p_te_cal' in globals() and p_te_cal is not None else p_te

w_out = w_te if 'w_te' in globals() else np.ones(len(df_te), dtype=float)

preds_A = pd.DataFrame({
    "SEQN": df_te["SEQN"].astype(int),
    "y": pd.Series(y_te, index=df_te.index, dtype=int),
    "p": pd.Series(p_out, index=df_te.index, dtype=float),        
    "w": pd.Series(w_out, index=df_te.index, dtype=float)
})

if 'p_te_cal' in globals() and p_te_cal is not None:
    preds_A["p_uncal"] = pd.Series(p_te, index=df_te.index, dtype=float)
    preds_A["p_cal"]   = pd.Series(p_te_cal, index=df_te.index, dtype=float)

for col in ["SDMVSTRA","SDMVPSU"]:
    if col in df_te.columns:
        preds_A[col] = df_te[col].astype(str)   
preds_A.to_csv("modelA_preds.csv", index=False)
print("Saved: modelA_preds.csv")

assert len(preds_A) == len(df_te), "Row count mismatch"
assert preds_A.notna().all().all(), "NaNs detected in saved predictions"
